In [1]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import matplotlib.patches as mpatches
from grid_utils import *
from pedestrian_utils import *
from pedestrian import *
from simulation import *
from results_utils import *
from visualization import *

In [2]:
output_dir = r"C:\Users\doohu\Desktop\대학원\디트\1209_디트시뮬레이션_격자"

In [3]:
plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 경로 설정
area_shp_path = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326\신규대상지_도보이동가능지역.shp"
buildings_shp_path = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326\신규대상지_건물지역_대피소제외.shp"
special_area_shp_path = r"C:\Users\doohu\Desktop\대학원\디트\디트파이썬코드\시장영역.shp"
evacuation_target_area_shp_path = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326\대피대상지_shp_point아님.shp"
creation_area_path = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326\agent_초기위치형성용.shp"

# GeoDataFrame 불러오기 및 좌표계 변환
target_crs = "EPSG:5181"
area_gdf = gpd.read_file(area_shp_path).to_crs(target_crs)
buildings_gdf = gpd.read_file(buildings_shp_path).to_crs(target_crs)
special_area_gdf = gpd.read_file(special_area_shp_path).to_crs(target_crs)
evacuation_target_area_gdf = gpd.read_file(evacuation_target_area_shp_path).to_crs(target_crs)
creation_area_gdf = gpd.read_file(creation_area_path).to_crs(target_crs)

print("Reprojection complete. All data is now in EPSG:5181.")

Reprojection complete. All data is now in EPSG:5181.


In [4]:
import numpy as np
import geopandas as gpd
from pedestrian_utils import generate_pedestrians

# 저장된 격자와 좌표 불러오기
output_dir = r"C:\Users\doohu\Desktop\대학원\디트\1209_디트시뮬레이션_격자"

grid = np.load(os.path.join(output_dir, "grid.npy"))
x_coords = np.loadtxt(os.path.join(output_dir, "x_coords.csv"), delimiter=",")
y_coords = np.loadtxt(os.path.join(output_dir, "y_coords.csv"), delimiter=",")

# GeoDataFrame 불러오기 및 좌표계 변환
special_area_shp_path = r"C:\Users\doohu\Desktop\대학원\디트\디트파이썬코드\시장영역.shp"
creation_area_path = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326\agent_초기위치형성용.shp"

special_area_gdf = gpd.read_file(special_area_shp_path).to_crs("EPSG:5181")
creation_area_gdf = gpd.read_file(creation_area_path).to_crs("EPSG:5181")

In [5]:
creation_area_cells= get_available_cells(grid, x_coords, y_coords, creation_area_gdf)
special_area_cells= get_available_cells(grid, x_coords, y_coords, special_area_gdf)
evacuation_target_area_cells= get_available_cells(grid, x_coords, y_coords, evacuation_target_area_gdf)
# special_area_cells와 general_area_cells를 리스트로 변환
special_area_cells=list(special_area_cells)
general_area_cells = list(creation_area_cells)
evacuation_target_area_cells= list(evacuation_target_area_cells)

In [ ]:
import pickle
import os

# 저장할 디렉터리 및 캐시 파일 경로 설정
cache_dir = r"C:\Users\doohu\Desktop\대학원\디트\1106_디트_최종대상지_4326"
cache_file = os.path.join(cache_dir, "area_cells_cache.pkl")

# 데이터 생성 함수
def generate_and_save_area_cells(grid, x_coords, y_coords, creation_area_gdf, special_area_gdf, evacuation_target_area_gdf):
    # get_available_cells를 이용해 셀 생성
    creation_area_cells = get_available_cells(grid, x_coords, y_coords, creation_area_gdf)
    special_area_cells = get_available_cells(grid, x_coords, y_coords, special_area_gdf)
    evacuation_target_area_cells = get_available_cells(grid, x_coords, y_coords, evacuation_target_area_gdf)
    
    # 리스트로 변환
    creation_area_cells = list(creation_area_cells)
    special_area_cells = list(special_area_cells)
    evacuation_target_area_cells = list(evacuation_target_area_cells)

    # 데이터 저장
    with open(cache_file, "wb") as f:
        pickle.dump({
            "creation_area_cells": creation_area_cells,
            "special_area_cells": special_area_cells,
            "evacuation_target_area_cells": evacuation_target_area_cells
        }, f)
    
    print("Data successfully generated and saved.")

# 데이터 불러오기 함수
def load_area_cells():
    if os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            cached_data = pickle.load(f)
            
            creation_area_cells = cached_data["creation_area_cells"]
            special_area_cells = cached_data["special_area_cells"]
            evacuation_target_area_cells = cached_data["evacuation_target_area_cells"]
            general_area_cells = creation_area_cells  # general_area_cells는 creation_area_cells와 동일
            
            print("Data successfully loaded.")
            
            return creation_area_cells, special_area_cells, evacuation_target_area_cells, general_area_cells
    else:
        print("Cache file not found. Please generate and save the data first.")
        return None, None, None, None

# 예제 코드 실행
# grid, x_coords, y_coords, creation_area_gdf, special_area_gdf, evacuation_target_area_gdf가 정의되어 있어야 합니다.
# 데이터 생성 및 저장
generate_and_save_area_cells(grid, x_coords, y_coords, creation_area_gdf, special_area_gdf, evacuation_target_area_gdf)

# 데이터 불러오기
creation_area_cells, special_area_cells, evacuation_target_area_cells, general_area_cells = load_area_cells()

# 확인용 출력
print("Creation Area Cells:", creation_area_cells)
print("Special Area Cells:", special_area_cells)
print("Evacuation Target Area Cells:", evacuation_target_area_cells)
print("General Area Cells:", general_area_cells)

# 이동 가능 격자 만들기

In [ ]:
import numpy as np

def save_walkable_grid(walkable_grid, file_path):
    """
    walkable_grid를 파일로 저장합니다.

    Parameters:
        walkable_grid (numpy.ndarray): 이동 가능한 셀이 0인 그리드
        file_path (str): 저장할 파일 경로
    """
    np.save(file_path, walkable_grid)
    print(f"walkable_grid가 '{file_path}'에 저장되었습니다.")

def load_walkable_grid(file_path):
    """
    파일에서 walkable_grid를 불러옵니다.

    Parameters:
        file_path (str): 불러올 파일 경로

    Returns:
        numpy.ndarray: 불러온 walkable_grid
    """
    walkable_grid = np.load(file_path)
    print(f"walkable_grid가 '{file_path}'에서 불러와졌습니다.")
    return walkable_grid

# 예제 사용법
if __name__ == "__main__":
    # 예제 walkable_grid 생성 (테스트용)
    walkable_grid = np.array([[0, 1, 0],
                              [0, 0, 1],
                              [1, 0, 0]])

    # 파일 경로 설정
    file_path = "walkable_grid.npy"

    # walkable_grid 저장
    save_walkable_grid(walkable_grid, file_path)

    # walkable_grid 불러오기
    loaded_walkable_grid = load_walkable_grid(file_path)

    # 불러온 walkable_grid 확인
    print(loaded_walkable_grid)
